In [4]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer

train_features = pd.read_csv('training_set_features.csv')
train_labels = pd.read_csv('training_set_labels.csv')
test_features = pd.read_csv('test_set_features.csv')
submission_format = pd.read_csv('submission_format.csv')

# Rename the column in submission_format
submission_format.rename(columns={'h1n1_vaccine': 'xyz_vaccine'}, inplace=True)

# merge the files
train_data = pd.merge(train_features, train_labels, on='respondent_id')
test_data = pd.merge(test_features, submission_format[['respondent_id']], on='respondent_id')

# Drop NA values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

train_data.head()
test_data.head()
                         

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird
5,26712,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Own,Employed,mlyzmhmf,"MSA, Principle City",0.0,2.0,fcxhlnwr,cmhcxjea
11,26718,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"> $75,000",Not Married,Own,Employed,fpwskwrf,"MSA, Principle City",0.0,0.0,arjwrbjb,hfxkjkmi


In [6]:
missing_values = test_data.isna().sum()
print(missing_values)

respondent_id                  0
xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_re

In [9]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# categorical_features = ['race', 'sex', 'marital_status', 'rent_or_own', 'employment_status',
#                         'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
# numerical_features = ['age_group', 'education', 'income_poverty', 'household_adults', 'household_children']

# categorical_data = train_data[categorical_features]
# categorical_data = test_data[categorical_features]


# ordinal_encoder = LabelEncoder()
# categorical_data['marital_status'] = ordinal_encoder.fit_transform(categorical_data['marital_status'])
# one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
# categorical_data = one_hot_encoder.fit_transform(categorical_data)
# categorical_data = pd.DataFrame(categorical_data.toarray(), columns=one_hot_encoder.get_feature_names_out())

# encoded_data = pd.concat([categorical_data, train_data[numerical_features]], axis=1)
# encoded_data = pd.concat([categorical_data, test_data[numerical_features]], axis=1)

<ipython-input-9-aeeed0040f98>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_data['marital_status'] = ordinal_encoder.fit_transform(categorical_data['marital_status'])


In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin

class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.label_encoders = {}
        self.columns = columns

    def fit(self, X, y=None):
        if self.columns is not None:
            for i, col in enumerate(self.columns):
                le = LabelEncoder()
                le.fit(X[:, i])
                self.label_encoders[col] = le
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.columns is not None:
            for i, col in enumerate(self.columns):
                X_copy[:, i] = self.label_encoders[col].transform(X[:, i])
        return X_copy


# Define the feature groups based on the provided column information
binary_cols = [
    'behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
    'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face', 'doctor_recc_xyz',
    'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months', 'health_worker', 'health_insurance',
    'sex', 'marital_status', 'rent_or_own'
]

ordinal_cols = [
    'xyz_concern', 'xyz_knowledge', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'opinion_xyz_sick_from_vacc',
    'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group', 'education',
    'income_poverty', 'employment_status', 'household_adults', 'household_children'
]

nominal_cols = [
    'race', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation'
]

# Define the preprocessing pipelines for each feature group
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('label_encoder', CustomLabelEncoder(columns=binary_cols))
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('label_encoder', CustomLabelEncoder(columns=ordinal_cols))
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_cols),
        ('ordinal', ordinal_transformer, ordinal_cols),
        ('nominal', nominal_transformer, nominal_cols)
    ]
)



In [12]:
# Split the training data into features and labels
X_train = train_data.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], axis=1)
y_train_xyz = train_data['xyz_vaccine']
y_train_seasonal = train_data['seasonal_vaccine']
X_test = test_data.drop(['respondent_id'], axis=1)

In [13]:
from sklearn.naive_bayes import GaussianNB
# Create the model pipeline
model_xyz = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', GaussianNB())])

model_seasonal = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', GaussianNB())])

# Train the models
model_xyz.fit(X_train, y_train_xyz)
model_seasonal.fit(X_train, y_train_seasonal)

# Predict probabilities for the test set
xyz_probabilities = model_xyz.predict_proba(X_test)[:, 1]
seasonal_probabilities = model_seasonal.predict_proba(X_test)[:, 1]


In [15]:
# Create the submission DataFrame
submission = pd.DataFrame({
    'respondent_id': test_data['respondent_id'],
    'xyz_vaccine': xyz_probabilities,
    'seasonal_vaccine': seasonal_probabilities
})

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!


In [19]:

df = pd.read_csv('submission.csv')

# Round off the values in the 'xyz_vaccine' and 'seasonal_vaccine' columns
df['xyz_vaccine'] = df['xyz_vaccine'].round(4)
df['seasonal_vaccine'] = df['seasonal_vaccine'].round(4)

# Save the updated DataFrame to a new CSV file
df.to_csv('submission_rounded.csv', index=False)

In [35]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Load the true labels and predictions
true_labels = pd.read_csv('submission_format.csv')
predictions = pd.read_csv('submission_rounded.csv')

true_labels_1 = true_labels['xyz_true_vaccine'].round().astype(int)
true_labels_2 = true_labels['seasonal_true_vaccine'].round().astype(int)
predictions_1 = predictions['xyz_vaccine'].round().astype(int)
predictions_2 = predictions['seasonal_vaccine'].round().astype(int)

# Calculate ROC AUC scores for both target variables
roc_auc_1 = roc_auc_score(true_labels_1, predictions_1)
roc_auc_2 = roc_auc_score(true_labels_2, predictions_2)

# Calculate the mean ROC AUC score
mean_roc_auc = (roc_auc_1 + roc_auc_2) / 2

print(f"ROC AUC for xyz_vaccine: {roc_auc_1}")
print(f"ROC AUC for seasonal_vaccine: {roc_auc_2}")
print(f"Mean ROC AUC: {mean_roc_auc}")


<class 'pandas.errors.IntCastingNaNError'>: Cannot convert non-finite values (NA or inf) to integer